In [1]:
import pandas as pd
from collections import Counter
import itertools
import warnings
warnings.filterwarnings('ignore')
import numpy as np 
from functools import reduce
import hashlib
import random
import matplotlib.pyplot as plt

In [3]:
com_3 = pd.read_csv('com402-3.csv',header=None,quotechar='"',skipinitialspace=True)
imdb_3 = pd.read_csv('imdb-3.csv',header=None,quotechar='"',skipinitialspace=True)
com_3.columns = ['email_salt','movie_salt','date','rating']
imdb_3.columns = ['email','movie','date','rating']

In [4]:
com_3['date'] = pd.to_datetime(com_3['date'], dayfirst=True)
com_3['min_date'] = com_3['date'] - pd.to_timedelta(13, unit='d')
com_3['max_date'] = com_3['date'] + pd.to_timedelta(13, unit='d')

imdb_3['date'] = pd.to_datetime(imdb_3['date'],dayfirst=True)

In [29]:
random_emails = ['keshav.singh@epfl.ch'] + list(imdb_3.email.unique()[0:35])

In [31]:
def iterator_movie_dict(com_3,my_subset):

    within_my_dates = pd.DataFrame(columns=['email_salt','movie_salt','date','rating','min_date','max_date','imdb_date','imdb_rating','imdb_movie'])
    for index, row in com_3.iterrows():

        for index_2, row_2 in my_subset.iterrows():

            if row.min_date <= row_2.date <= row.max_date:

                within_my_dates = within_my_dates.append({'email_salt':row.email_salt,
                                                          'movie_salt':row.movie_salt,
                                                          'date':row.date,
                                                          'rating':row.rating,
                                                          'min_date':row.min_date,
                                                          'max_date':row.max_date,
                                                          'imdb_date': row_2.date,
                                                          'imdb_rating': row_2.rating,
                                                          'imdb_movie': row_2.movie},ignore_index=True)

    same_ratings = within_my_dates.loc[within_my_dates['rating'] == within_my_dates['imdb_rating']]
    my_hash = same_ratings['email_salt'].value_counts().index[0]
    print('new iter')
    #print(same_ratings)
    print(same_ratings['email_salt'].value_counts()[0])
    my_movies = same_ratings.loc[same_ratings['email_salt']==my_hash]
    my_movies = my_movies.drop_duplicates(subset=['imdb_movie'],keep=False).sort_values(by='imdb_movie')

    dict_movies = pd.Series(my_movies.movie_salt.values,index=my_movies.imdb_movie).to_dict()

    return my_hash, dict_movies

In [35]:
email_hashes = {}
movie_hashes = []

for email_ in random_emails:
    
    
    my_subset = imdb_3.loc[imdb_3['email']==email_]
    
    my_hash, dict_movie = iterator_movie_dict(com_3,my_subset)
    
    print("#########")
    print("The e-mail: {} , The hash: {}".format(email_,my_hash))
    
    email_hashes[email_] = my_hash
    movie_hashes.append(dict_movie)

new iter
52
#########
The e-mail: keshav.singh@epfl.ch , The hash: 34510d39c13a0ecbe5658e22eb1f1c4d0fa7d9f845cb44fa4723e4faefee9e68
new iter
56
#########
The e-mail: fares.ahmed@epfl.ch , The hash: 829e26707476b1601b653c675855441b7a1f94b941f6cf1b1823ab363922d47c
new iter
53
#########
The e-mail: isabelle.pumford@epfl.ch , The hash: af91ab607d6011391b444dd5fd4a7ef6b09d35d7d743ec03eb49b349076c6f49
new iter
52
#########
The e-mail: nourchene.benromdhane@epfl.ch , The hash: 966716ffea35903896d5f81d2f3c2e6f4b531d2cccd265efe47c3a61728fb10b
new iter
54
#########
The e-mail: damian.dudzicz@epfl.ch , The hash: 1f3ddf517ce65c01588667da52aaf1cf55d39444151e391911b2b2ab360ee894
new iter
52
#########
The e-mail: amandine.evard@epfl.ch , The hash: 79fe357a3eb9e8e0226530fe857507d6a0449dbf73b72e96a2c525417de8ec72
new iter
54
#########
The e-mail: loic.serafin@epfl.ch , The hash: 3fb35f074870b4159c3360b7db69c16367e2a8e41826bf3edb5713618ce4045f
new iter
52
#########
The e-mail: jelena.banjac@epfl.ch , Th

In [36]:
final_dict = {}
for d in movie_hashes: 
    final_dict.update(d)
    
my_hash = email_hashes['keshav.singh@epfl.ch']
my_movies = com_3.loc[com_3['email_salt']== my_hash]

list_of_movies = []

for movie in my_movies.movie_salt.values:
    for key_, value_ in final_dict.items():
        if movie == value_:
            list_of_movies.append(key_)

In [39]:
file = open("movies_c.txt","w")
for movie_ in list_of_movies:
    file.write(movie_ + "\n")
    
file.close()

In [40]:
len(list_of_movies)

100

In [26]:
com_3.head()


,email_salt,movie_salt,date,rating,min_date,max_date
0,05aa2e00cb70a4bfee402474d299c60f77826f55f66a2b...,019640e64091ecef31e94b11f4a4d0c8c897dbc1e85334...,2001-01-01,4,2000-12-19,2001-01-14
1,d64701e7ca92830d3cbbd0558f87f3f7eabb9ac9e822e7...,ddff9ab04486f5990c106a17e90352b1ae5d8cd30e4bfe...,2001-01-01,1,2000-12-19,2001-01-14
2,ac64946bd26fc2bf51ed2173347ef02f75c828aab2f36f...,0a96c7b53dd6efb8231e4aed25daa8116a2d4f8b28cc51...,2001-01-01,3,2000-12-19,2001-01-14
3,317356d2ad86c92dc75bfbbb95e9b5a4169a5f36bb413c...,8a208a550464b4d0f6cd7cdc3e0d17a1cdc3046705fa34...,2001-01-01,2,2000-12-19,2001-01-14
4,df647173915c8bf6ea4833ee5f8006a61bf531120eaa6a...,5739b9db30f30d0a7e89492bc61f531fd764744dabcd9a...,2002-01-01,3,2001-12-19,2002-01-14


In [27]:
imdb_3.head()

,email,movie,date,rating
0,fares.ahmed@epfl.ch,The Graduate,2000-01-01,2
1,isabelle.pumford@epfl.ch,Annie Hall,2001-01-01,4
2,nourchene.benromdhane@epfl.ch,The Diving Bell and the Butterfly,2002-01-01,3
3,damian.dudzicz@epfl.ch,A Man Escaped,2002-01-01,3
4,amandine.evard@epfl.ch,The Army of Shadows,2002-01-01,1
